In [1]:
import mysql.connector
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import math
import os
import sys
import logging
mf_module_path = os.path.abspath(os.path.join('../python'))
if mf_module_path not in sys.path:
    sys.path.append(mf_module_path)
import mf
import mf_random
import hpoutil
import networkx
import obonet
import pickle

**Connect to MySQL database**

In [150]:
mydb = mysql.connector.connect(host='localhost',
                               user='mimicuser',
                               passwd='mimic',
                               database='mimiciiiv13',
                              auth_plugin='mysql_native_password')

First approach to query mysql from python

Check that MySQL connection works properly

In [151]:
df = pd.read_sql_query("SELECT * FROM LABEVENTS LIMIT 5;", mydb)
df

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,1,2,163353,51143,2138-07-17 20:48:00,0,0.0,%,None
1,2,2,163353,51144,2138-07-17 20:48:00,0,0.0,%,None
2,3,2,163353,51146,2138-07-17 20:48:00,0,0.0,%,None
3,4,2,163353,51200,2138-07-17 20:48:00,0,0.0,%,None
4,5,2,163353,51221,2138-07-17 20:48:00,0,0.0,%,abnormal


Get a cursor so that it can be used later

In [152]:
cursor = mydb.cursor(buffered=True)

We explored several method to compute the synergy score for different diseases. Method 1-3 all worked but the time and space requirements are too high. See the archived file. Here, we use method 4 to compute phenotype pairwise synergies.


# Synergy between Lab-derived Abnormalities

## Algorithm

This method relies on the power of MySQL for doing queies and joins, return a batch of phenotype profiles a time, and then use the power of Numpy to do numeric computation.

Specificially, the method runs the following algorithm:

    1. For one diagnosis code, specify the phenotypes to analyze--a list of HPO terms.
    2. For a batch of patient*encounters, return a list of diagnosis codes (1 or 0)
    3. For the same batch of patient*encounters, return a list of phenotypes.
    4. Create a numpy array with dimension (N x P)
    5. Perform numeric computation with Numpy:
        outer product for ++ of PxP.T
        outer product for +- of Px(1-P).T
        outer product for -+ of (1-P)xP
        outer product for -- of (1-P)x(1-P).T
        combine the above with - and + of diagnosis value
        stack them together as a (N x P x P x 8) matrix.
        Step 1 - 5 are performed at each site. The resulting matrix is returned to JAX for final analyze.
    6. Compute pairwise synergy:
        use the multi-dimension array to calculate p(D = 1), p(D = 0), p(P1 * P2)
        compute mutual information of each phenotype in regarding to one diagnosis I(P:D)
        compute mutual information of two phenotypes in regarding to one diagnosis I(P:D)
        compute pairwise synergy
        

In [5]:
#TODO: rewrite to be backward compatible
def diagnosis_set():
    """Aggregate ICD9 codes with the first three digit and count how many times they appear. 
    Note this function uses encounters as the unit, meaning a code will counted twice if same patient was 
    diagnosed again at a later encounter."""
    diagnosis_count = pd.read_sql_query("SELECT SUBJECT_ID, HADM_ID, \
        CASE \
        WHEN(ICD9_CODE LIKE 'V%') THEN SUBSTRING(ICD9_CODE, 1, 3) \
        WHEN(ICD9_CODE LIKE 'E%') THEN SUBSTRING(ICD9_CODE, 1, 4) \
        ELSE SUBSTRING(ICD9_CODE, 1, 3) END AS ICD9 \
        FROM DIAGNOSES_ICD", mydb)
    diagnosisSet = diagnosis_count.drop_duplicates().groupby('ICD9').size().sort_values(ascending=False)
    return diagnosisSet

#TODO: rewrite to be backward compatible
def createAbnormalPhenotypeTable(threshold, include_inferred=True, force_update=True):
    """
    This is the abnormal phenotypes. 
    @include_inferred whether to include inferred HPO. Default true.
    @force_update whether current table, if present, should be forced to update
    """
    if force_update:
        cursor.execute('''DROP TEMPORARY TABLE IF EXISTS p''')
    if include_inferred:
        cursor.execute('''
                    CREATE TEMPORARY TABLE IF NOT EXISTS p
                    WITH abnorm AS (
                        SELECT
                            LABEVENTS.SUBJECT_ID, LABEVENTS.HADM_ID, LabHpo.MAP_TO
                        FROM 
                            LABEVENTS 
                        JOIN LabHpo on LABEVENTS.ROW_ID = LabHpo.ROW_ID
                        WHERE LabHpo.NEGATED = 'F'
                        
                        UNION ALL
                        
                        SELECT 
                            LABEVENTS.SUBJECT_ID, LABEVENTS.HADM_ID, INFERRED_LABHPO.INFERRED_TO AS MAP_TO 
                        FROM 
                            INFERRED_LABHPO 
                        JOIN 
                            LABEVENTS ON INFERRED_LABHPO.LABEVENT_ROW_ID = LABEVENTS.ROW_ID
                        )
                    SELECT SUBJECT_ID, HADM_ID, MAP_TO
                    FROM abnorm 
                    GROUP BY SUBJECT_ID, HADM_ID, MAP_TO
                    HAVING COUNT(*) > {}
                    -- parameter to control how to define an abnormal phenotype is present.
                '''.format(threshold))
    else:       
        cursor.execute('''
                    CREATE TEMPORARY TABLE IF NOT EXISTS p
                    WITH abnorm AS (
                        SELECT
                            LABEVENTS.SUBJECT_ID, LABEVENTS.HADM_ID, LabHpo.MAP_TO
                        FROM 
                            LABEVENTS 
                        JOIN LabHpo on LABEVENTS.ROW_ID = LabHpo.ROW_ID
                        WHERE LabHpo.NEGATED = 'F')
                    SELECT SUBJECT_ID, HADM_ID, MAP_TO
                    FROM abnorm 
                    GROUP BY SUBJECT_ID, HADM_ID, MAP_TO
                    HAVING COUNT(*) > {}
                    -- parameter to control how to define an abnormal phenotype is present.
                '''.format(threshold))
    cursor.execute('CREATE INDEX p_idx01 ON p (SUBJECT_ID, HADM_ID)')
    cursor.execute('CREATE INDEX p_idx02 ON p (MAP_TO);')


#TODO: rewrite to be backward compatible
def encountersWithDiagnosis(diagnosis):
    cursor.execute('''DROP TEMPORARY TABLE IF EXISTS d''')
    cursor.execute('''
        CREATE TEMPORARY TABLE IF NOT EXISTS d
        SELECT 
            DISTINCT SUBJECT_ID, HADM_ID, 1 AS DIAGNOSIS
        FROM 
            DIAGNOSES_ICD 
        WHERE ICD9_CODE LIKE '{}%'
        -- This is encounters with positive diagnosis
    '''.format(diagnosis))
    cursor.execute('CREATE INDEX d_idx01 ON d(SUBJECT_ID, HADM_ID)')

    
def createPhenotypeSet(diagnosis, threshold=1000):
    """
    Create the phenotypes that we should analyze. Exemely less frequently observed phenotypes are excluded.
    """
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS ps')
    cursor.execute('''
            CREATE TEMPORARY TABLE ps
            WITH pd AS(
                SELECT p.*
                FROM 
                    p JOIN (SELECT 
                                DISTINCT SUBJECT_ID, HADM_ID, 1 AS DIAGNOSIS
                            FROM 
                                DIAGNOSES_ICD 
                            WHERE ICD9_CODE LIKE '{}%') AS d
                    ON p.SUBJECT_ID = d.SUBJECT_ID AND p.HADM_ID = d.HADM_ID)
            SELECT 
                MAP_TO, COUNT(*) AS N, 1 AS PHENOTYPE
            FROM pd
            GROUP BY MAP_TO
            HAVING N > {}
            ORDER BY N DESC'''.format(diagnosis, threshold))
    phenoSet = pd.read_sql_query('SELECT * FROM ps', mydb)
    return phenoSet


def batch_query(start_index, end_index):
    batch_size_actual = pd.read_sql_query('''
                SELECT 
                    COUNT(DISTINCT SUBJECT_ID, HADM_ID) 
                FROM admissions 
                WHERE SUBJECT_ID BETWEEN {} AND {}
                '''.format(start_index, end_index), mydb).iloc[0,0]
    # create diagnosis table
    diagnosisList = pd.read_sql_query('''
                WITH a AS (
                    SELECT DISTINCT SUBJECT_ID, HADM_ID 
                    FROM admissions 
                    WHERE SUBJECT_ID BETWEEN {} AND {})
                SELECT 
                    a.SUBJECT_ID, a.HADM_ID, IF(d.DIAGNOSIS IS NULL, 0, 1) AS DIAGNOSIS
                FROM 
                    a
                LEFT JOIN
                    d ON a.SUBJECT_ID = d.SUBJECT_ID AND a.HADM_ID = d.HADM_ID         
                '''.format(start_index, end_index), mydb)
    # create phenotype profile table
    phenotyle_profile = pd.read_sql_query('''
        WITH 
            a AS (
                    SELECT 
                        DISTINCT SUBJECT_ID, HADM_ID 
                    FROM 
                        admissions 
                    WHERE SUBJECT_ID BETWEEN {} AND {}), 
            c as (
                SELECT a.*, ps.MAP_TO
                FROM a
                JOIN ps),
                -- cross product of all patient*encounter and phenotypes list
            pp as (
                SELECT p.*, 1 AS PHENOTYPE 
                FROM p RIGHT JOIN a 
                ON p.SUBJECT_ID = a.SUBJECT_ID AND p.HADM_ID = a.HADM_ID)

        SELECT c.SUBJECT_ID, c.HADM_ID, c.MAP_TO, IF(pp.PHENOTYPE IS NULL, 0, 1) AS PHENOTYPE 
        FROM pp 
        RIGHT JOIN c ON pp.SUBJECT_ID = c.SUBJECT_ID and pp.HADM_ID = c.HADM_ID AND pp.MAP_TO = c.MAP_TO
        '''.format(start_index, end_index), mydb)
    return batch_size_actual, diagnosisList, phenotyle_profile

In [78]:
def iterate_in_batch(logger):
    logger.info('starting iterate_in_batch()')
    batch_size = 100
    # find the set of diagnosis that are worthy to analyze
    diagnosisSet = diagnosis_set()
    logger.info('diagnosis set completed')

    # create a temp table for abnormal phenotypes of each patient*encounter that met the threshold
    #createAbnormalPhenotypeTable(threshold=1, force_update=True)
    logger.info('createAbnormalPhenotypeTable() completed')
    
    synergies = {}
    
    for diagnosis in diagnosisSet.keys():
        if (diagnosisSet[diagnosis] > 5000):
            # create a temp table for diagnosis of all patient*encouter to analyze
            encountersWithDiagnosis(diagnosis)
            logger.info('encountersWithDiagnosis() completed')

            ## create a list of phenotypes that we want to analyze for the specified disease and preset threshold
            phenoSet = createPhenotypeSet(diagnosis, threshold=100)
            logger.info('phenoSet completed')
            P_SIZE = len(phenoSet)

            ## find the start and end ROW_ID for patient*encounter
            ADM_ID_START, ADM_ID_END = pd.read_sql_query('SELECT MIN(ROW_ID) AS min, MAX(ROW_ID) AS max FROM admissions', mydb).iloc[0]
            batch_N = ADM_ID_END - ADM_ID_START + 1
            TOTAL_BATCH = math.ceil(batch_N / batch_size) # total number of batches
            synergies[diagnosis] = mf.Synergy(diagnosis, phenoSet.MAP_TO)
            logger.info('starting batch queries for {}'.format(diagnosis))
            for i in np.arange(TOTAL_BATCH):
                start_index = i * batch_size + ADM_ID_START
                if i < TOTAL_BATCH - 1:
                    end_index = start_index + batch_size - 1
                else:
                    end_index = batch_N
                
                batch_size_actual, diagnosisList, phenotyle_profile = batch_query(start_index, end_index)
                
                if batch_size_actual > 0 :
                    diagnosisVector = diagnosisList.DIAGNOSIS
                    phenotypeProfileMatrix = phenotyle_profile.PHENOTYPE.values.reshape([batch_size_actual, P_SIZE])
                    if i % 100 == 0:
                        logger.info('new batch: start_index={}, end_index={}, batch_size= {}, phenotype_size = {}'.format(start_index, end_index, batch_size_actual, len(phenoSet)))
                    synergies[diagnosis].add_batch(phenotypeProfileMatrix, diagnosisVector)
    
    return synergies

It takes about 10 minutes to set up the phenotype table (p). Afterward, each disease takes about 10 minutes to complete the summary statistics.

In [ ]:
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',level=logging.DEBUG, stream=sys.stdout)
logger.setLevel(logging.DEBUG)

start = datetime.datetime.now()

synergies = iterate_in_batch(logger)
   
end = datetime.datetime.now()


In [77]:
print('running time: {}s'.format((end - start).total_seconds()))

In [92]:
with open('synergies.obj', 'wb') as synergies_file:
    pickle.dump(synergies, synergies_file)

close database connection

In [ ]:
cursor.close()
mydb.close()

# Synergy between lab-derived and radiology report-derived Abnormalities

## Algorithm

The algorithm is about the same with the Method 3 in mutual_info_archive. Briefly, 

    * Select encounterOfInterest, temp table: JAX_encounterOfInterest(SUBJECT_ID, HADM_ID)
    * Init diagnosisProfile: temp table: JAX_diagnosisProfile(SUBJECT_ID, HADM_ID, ICD, N)
    * Init textHpoProfile: temp table: JAX_textHpoProfile(SUBJECT_ID, HADM_ID, MAP_TO, N)
    * Init labHpoProfile: temp table: JAX_labHpoProfile(SUBJECT_ID, HADM_ID, MAP_TO, N)
    
    * Rank ICD frequency, temp table: JAX_diagFrequencyRank(ICD, N)
      select diagOfInterest
    * Rank textHPO frequency, temp table: JAX_textHpoFrequencyRank(MAP_TO, N)
      select textHpoOfInterest
    * Rank labHPO frequency, temp table: JAX_labHpoFrequencyRank(MAP_TO, N)
      select labHpoOfInterest
    
    * Iteratation
      for diagnosis in diagOfInterest
          for textHpo in textHpoOfInterest
              for labHpo in labHpoOfInterest
                 Assign diagnosis value: assignDiagnosis(), table: (SUBJECT_ID, HADM_ID, DIAGNOSIS)
                 Assign text2hpo phenotype value: table: SUBJECT_ID, HADM_ID, PHEN_TEXT
                 Assign lab2hpo phenotype value: table: SUBJECT_ID, HADM_ID, PHEN_LAB

In [153]:
# define encounters of interest
def encounterOfInterest(debug=False):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_encounterOfInterest')
    if debug:
        limit = 'LIMIT 100'
    else:
        limit = ''
    # This is admissions that we want to analyze, 'LIMIT 100' in debug mode
    cursor.execute('''
                CREATE TEMPORARY TABLE IF NOT EXISTS JAX_encounterOfInterest 
                SELECT 
                    DISTINCT SUBJECT_ID, HADM_ID 
                FROM admissions
                {}
                '''.format(limit))
    
def diagnosisProfile():
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_diagnosisProfile')
    cursor.execute('''
                CREATE TEMPORARY TABLE IF NOT EXISTS JAX_diagnosisProfile
                SELECT 
                    DIAGNOSES_ICD.SUBJECT_ID, DIAGNOSES_ICD.HADM_ID, DIAGNOSES_ICD.ICD9_CODE
                FROM
                    DIAGNOSES_ICD
                RIGHT JOIN
                    JAX_encounterOfInterest
                ON 
                    DIAGNOSES_ICD.SUBJECT_ID = JAX_encounterOfInterest.SUBJECT_ID 
                    AND 
                    DIAGNOSES_ICD.HADM_ID = JAX_encounterOfInterest.HADM_ID
                ''')
    
def textHpoProfile(include_inferred=True, threshold=1):
    if include_inferred:
        cursor.execute('''
                    CREATE TEMPORARY TABLE IF NOT EXISTS JAX_textHpoProfile
                    WITH abnorm AS (
                        SELECT
                            NOTEEVENTS.SUBJECT_ID, NOTEEVENTS.HADM_ID, NoteHpoClinPhen.MAP_TO
                        FROM 
                            NOTEEVENTS 
                        JOIN NoteHpoClinPhen on NOTEEVENTS.ROW_ID = NoteHpoClinPhen.NOTES_ROW_ID
                        
                        UNION ALL
                        
                        SELECT
                            NOTEEVENTS.SUBJECT_ID, NOTEEVENTS.HADM_ID, Inferred_NoteHpo.INFERRED_TO AS MAP_TO
                        FROM 
                            NOTEEVENTS 
                        JOIN Inferred_NoteHpo on NOTEEVENTS.ROW_ID = Inferred_NoteHpo.NOTEEVENT_ROW_ID
                        )
                    SELECT SUBJECT_ID, HADM_ID, MAP_TO, 1 AS dummy
                    FROM abnorm 
                    GROUP BY SUBJECT_ID, HADM_ID, MAP_TO
                    HAVING COUNT(*) > {}
                    -- parameter to control how to define an abnormal phenotype is present.
                '''.format(threshold))
        
    else:
        cursor.execute('''
                    CREATE TEMPORARY TABLE IF NOT EXISTS JAX_p_text
                    WITH abnorm AS (
                        SELECT
                            NOTEEVENTS.SUBJECT_ID, NOTEEVENTS.HADM_ID, NoteHpoClinPhen.MAP_TO
                        FROM 
                            NOTEEVENTS 
                        JOIN NoteHpoClinPhen on NOTEEVENTS.ROW_ID = NoteHpoClinPhen.NOTES_ROW_ID)
                    SELECT SUBJECT_ID, HADM_ID, MAP_TO
                    FROM abnorm 
                    GROUP BY SUBJECT_ID, HADM_ID, MAP_TO, 1 AS dummy
                    HAVING COUNT(*) > {}
                    -- parameter to control how to define an abnormal phenotype is present.
                '''.format(threshold))
        
def indexTextHpoProfile():
    cursor.execute('CREATE INDEX JAX_textHpoProfile_idx01 ON JAX_textHpoProfile (SUBJECT_ID, HADM_ID)')
    cursor.execute('CREATE INDEX JAX_textHpoProfile_idx02 ON JAX_textHpoProfile (MAP_TO);')
    cursor.execute('CREATE INDEX JAX_textHpoProfile_idx03 ON JAX_textHpoProfile (SUBJECT_ID, HADM_ID, MAP_TO)')
    
def labHpoProfile(threshold, include_inferred=True, force_update=True):
    # TODO: refactor the method 
    #createAbnormalPhenotypeTable(threshold, include_inferred=True, force_update=True)
    
    if force_update:
        cursor.execute('''DROP TEMPORARY TABLE IF EXISTS JAX_labHpoProfile''')
    if include_inferred:
        cursor.execute('''
                    CREATE TEMPORARY TABLE IF NOT EXISTS JAX_labHpoProfile
                    WITH abnorm AS (
                        SELECT
                            LABEVENTS.SUBJECT_ID, LABEVENTS.HADM_ID, LabHpo.MAP_TO
                        FROM 
                            LABEVENTS 
                        JOIN LabHpo on LABEVENTS.ROW_ID = LabHpo.ROW_ID
                        WHERE LabHpo.NEGATED = 'F'
                        
                        UNION ALL
                        
                        SELECT 
                            LABEVENTS.SUBJECT_ID, LABEVENTS.HADM_ID, INFERRED_LABHPO.INFERRED_TO AS MAP_TO 
                        FROM 
                            INFERRED_LABHPO 
                        JOIN 
                            LABEVENTS ON INFERRED_LABHPO.LABEVENT_ROW_ID = LABEVENTS.ROW_ID
                        )
                    SELECT SUBJECT_ID, HADM_ID, MAP_TO, 1 AS dummy
                    FROM abnorm 
                    GROUP BY SUBJECT_ID, HADM_ID, MAP_TO
                    HAVING COUNT(*) > {}
                    -- parameter to control how to define an abnormal phenotype is present.
                '''.format(threshold))
    else:       
        cursor.execute('''
                    CREATE TEMPORARY TABLE IF NOT EXISTS JAX_labHpoProfile
                    WITH abnorm AS (
                        SELECT
                            LABEVENTS.SUBJECT_ID, LABEVENTS.HADM_ID, LabHpo.MAP_TO
                        FROM 
                            LABEVENTS 
                        JOIN LabHpo on LABEVENTS.ROW_ID = LabHpo.ROW_ID
                        WHERE LabHpo.NEGATED = 'F')
                    SELECT SUBJECT_ID, HADM_ID, MAP_TO, 1 AS dummy
                    FROM abnorm 
                    GROUP BY SUBJECT_ID, HADM_ID, MAP_TO
                    HAVING COUNT(*) > {}
                    -- parameter to control how to define an abnormal phenotype is present.
                '''.format(threshold))

def indexLabHpoProfile():
    cursor.execute('CREATE INDEX JAX_labHpoProfile_idx01 ON JAX_labHpoProfile (SUBJECT_ID, HADM_ID)')
    cursor.execute('CREATE INDEX JAX_labHpoProfile_idx02 ON JAX_labHpoProfile (MAP_TO);')
    cursor.execute('CREATE INDEX JAX_labHpoProfile_idx03 ON JAX_labHpoProfile (SUBJECT_ID, HADM_ID, MAP_TO)')
    
def rankICD():
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_textHpoFrequencyRank')
    cursor.execute("""
        CREATE TEMPORARY TABLE IF NOT EXISTS JAX_diagFrequencyRank
        WITH JAX_temp_diag AS (
            SELECT DISTINCT SUBJECT_ID, HADM_ID, 
                CASE 
                    WHEN(ICD9_CODE LIKE 'V%') THEN SUBSTRING(ICD9_CODE, 1, 3) 
                    WHEN(ICD9_CODE LIKE 'E%') THEN SUBSTRING(ICD9_CODE, 1, 4) 
                ELSE 
                    SUBSTRING(ICD9_CODE, 1, 3) END AS ICD9_CODE 
            FROM JAX_diagnosisProfile)
        SELECT 
            ICD9_CODE, COUNT(*) AS N
        FROM
            JAX_temp_diag
        GROUP BY 
            ICD9_CODE
        ORDER BY N
        DESC
        """)

def rankHpoFromText(diagnosis):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_textHpoFrequencyRank')
    cursor.execute('''
            CREATE TEMPORARY TABLE JAX_textHpoFrequencyRank            
            WITH pd AS(
                SELECT 
                    JAX_textHpoProfile.*
                FROM 
                    JAX_textHpoProfile 
                JOIN (
                    SELECT 
                        DISTINCT SUBJECT_ID, HADM_ID
                    FROM 
                        JAX_diagnosisProfile 
                    WHERE 
                        ICD9_CODE LIKE '{}%') AS d
                ON 
                    JAX_textHpoProfile.SUBJECT_ID = d.SUBJECT_ID AND JAX_textHpoProfile.HADM_ID = d.HADM_ID)
            SELECT 
                MAP_TO, COUNT(*) AS N, 1 AS PHENOTYPE
            FROM pd
            GROUP BY MAP_TO
            ORDER BY N DESC'''.format(diagnosis))
    
def rankHpoFromLab(diagnosis):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_labHpoFrequencyRank')
    cursor.execute('''
            CREATE TEMPORARY TABLE JAX_labHpoFrequencyRank            
            WITH pd AS(
                SELECT 
                    JAX_labHpoProfile.*
                FROM 
                    JAX_labHpoProfile 
                JOIN (
                    SELECT 
                        DISTINCT SUBJECT_ID, HADM_ID
                    FROM 
                        JAX_diagnosisProfile 
                    WHERE 
                        ICD9_CODE LIKE '{}%') AS d
                ON 
                    JAX_labHpoProfile.SUBJECT_ID = d.SUBJECT_ID AND JAX_labHpoProfile.HADM_ID = d.HADM_ID)
            SELECT 
                MAP_TO, COUNT(*) AS N, 1 AS PHENOTYPE
            FROM pd
            GROUP BY MAP_TO
            ORDER BY N DESC'''.format(diagnosis))           

In [209]:
# assign 0 or 1 to each encouter whether a diagnosis is observed
def createDiagnosisTable(diagnosis):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag')
    cursor.execute('''
                CREATE TEMPORARY TABLE IF NOT EXISTS JAX_mf_diag 
                WITH 
                    d AS (
                        SELECT 
                            DISTINCT SUBJECT_ID, HADM_ID, '1' AS DIAGNOSIS
                        FROM 
                            JAX_diagnosisProfile 
                        WHERE ICD9_CODE LIKE '{}%')
                    -- This is encounters with positive diagnosis

                SELECT 
                    DISTINCT a.SUBJECT_ID, a.HADM_ID, IF(d.DIAGNOSIS IS NULL, '0', '1') AS DIAGNOSIS
                FROM 
                    JAX_encounterOfInterest AS a
                LEFT JOIN
                    d ON a.SUBJECT_ID = d.SUBJECT_ID AND a.HADM_ID = d.HADM_ID       
                /* -- This is the first join for diagnosis (0, or 1) */    
                '''.format(diagnosis))
    cursor.execute('CREATE INDEX JAX_mf_diag_idx01 ON JAX_mf_diag (SUBJECT_ID, HADM_ID)')

# assign 0 or 1 to each encounter whether a phenotype is observed from radiology reports
def diagnosisTextHpo(phenotype):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag_textHpo')
    """
    cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_textHpo
        SELECT 
            L.*, IF(R.MAP_TO IS NULL, '0', '1') AS PHEN_TXT
        FROM JAX_mf_diag AS L 
        LEFT JOIN 
            (SELECT * 
            FROM JAX_textHpoProfile 
            WHERE JAX_textHpoProfile.MAP_TO = '{}') AS R 
        ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID 
    '''.format(phenotype))
    """
    cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_textHpo
        WITH L AS (SELECT JAX_mf_diag.*, '{}' AS PHEN_TXT FROM JAX_mf_diag)
        SELECT 
            L.*, IF(R.dummy IS NULL, '0', '1') AS PHEN_TXT_VALUE
        FROM L 
        LEFT JOIN 
            JAX_textHpoProfile AS R
        ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID AND L.PHEN_TXT = R.MAP_TO
    '''.format(phenotype))
    cursor.execute('CREATE INDEX JAX_mf_diag_textHpo_idx01 ON JAX_mf_diag_textHpo (SUBJECT_ID, HADM_ID)')

def diagnosisAllTextHpo(threshold_min, threshold_max):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag_allTextHpo')
    cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_allTextHpo
        WITH 
            P AS (SELECT MAP_TO AS PHEN_TXT FROM JAX_textHpoFrequencyRank WHERE N BETWEEN {} AND {}),
            L AS (SELECT * FROM JAX_mf_diag JOIN P)
        SELECT 
            L.*, IF(R.dummy IS NULL, '0', '1') AS PHEN_TXT_VALUE
        FROM L 
        LEFT JOIN 
            JAX_textHpoProfile AS R
        ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID AND L.PHEN_TXT = R.MAP_TO
    '''.format(threshold_min, threshold_max))
    cursor.execute('CREATE INDEX JAX_mf_diag_allTextHpo_idx01 ON JAX_mf_diag_allTextHpo (SUBJECT_ID, HADM_ID, PHEN_TXT)')
    
def diagnosisLabHpo(phenotype):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag_labHpo')
    cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_labHpo
        WITH L AS (SELECT JAX_mf_diag.*, '{}' AS PHEN_LAB FROM JAX_mf_diag)
        SELECT 
            L.*, IF(R.dummy IS NULL, '0', '1') AS PHEN_LAB_VALUE
        FROM L 
        LEFT JOIN 
             JAX_labHpoProfile AS R 
        ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID AND L.PHEN_LAB = R.MAP_TO
    '''.format(phenotype))
    cursor.execute('CREATE INDEX JAX_mf_diag_labHpo_idx01 ON JAX_mf_diag_labHpo (SUBJECT_ID, HADM_ID)')
    
def diagnosisAllLabHpo(threshold_min, threshold_max):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag_allLabHpo')
    cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_allLabHpo
        WITH 
            P AS (SELECT MAP_TO AS PHEN_LAB FROM JAX_labHpoFrequencyRank WHERE N BETWEEN {} AND {}),
            L AS (SELECT * FROM JAX_mf_diag JOIN P)
        SELECT 
            L.*, IF(R.dummy IS NULL, '0', '1') AS PHEN_LAB_VALUE
        FROM L 
        LEFT JOIN 
             JAX_labHpoProfile AS R 
        ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID AND L.PHEN_LAB = R.MAP_TO
    '''.format(threshold_min, threshold_max))
    #cursor.execute('CREATE INDEX JAX_mf_diag_labHpo_idx01 ON JAX_mf_diag_labHpo (SUBJECT_ID, HADM_ID)')

def diagnosisTextLab(phenotype):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag_txtHpo_labHpo')
    result = cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_txtHpo_labHpo 
        WITH L AS (SELECT JAX_mf_diag_textHpo.*, '{}' AS PHEN_LAB FROM JAX_mf_diag_textHpo)
        SELECT L.*, IF(R.dummy IS NULL, '0', '1') AS PHEN_LAB_VALUE
        FROM L 
        LEFT JOIN 
            JAX_labHpoProfile AS R 
        ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID AND L.PHEN_LAB = R.MAP_TO
    '''.format(phenotype), mydb)
    
    
def diagnosisAllTextAllLab(threshold_min, threshold_max):
    cursor.execute('DROP TEMPORARY TABLE IF EXISTS JAX_mf_diag_allTxtHpo_allLabHpo')
    cursor.execute('''
        CREATE TEMPORARY TABLE JAX_mf_diag_allTxtHpo_allLabHpo 
        WITH 
            P AS (SELECT MAP_TO AS PHEN_LAB, NULL AS PHEN_LAB_VALUE FROM JAX_labHpoFrequencyRank WHERE N BETWEEN {} AND {})
        SELECT * FROM JAX_mf_diag_allTextHpo JOIN P
    '''.format(threshold_min, threshold_max), mydb)
    cursor.execute('CREATE INDEX JAX_mf_diag_allTxtHpo_allLabHpo_idx01 ON JAX_mf_diag_allTxtHpo_allLabHpo (SUBJECT_ID, HADM_ID, PHEN_LAB)')
    
    cursor.execute('''
        UPDATE JAX_mf_diag_allTxtHpo_allLabHpo 
        SET PHEN_LAB_VALUE = (
            SELECT IF(R.dummy IS NULL, '0', '1')
            FROM JAX_mf_diag_allTxtHpo_allLabHpo AS L 
            LEFT JOIN 
                JAX_labHpoProfile AS R 
            ON L.SUBJECT_ID = R.SUBJECT_ID AND L.HADM_ID = R.HADM_ID AND L.PHEN_LAB = R.MAP_TO
        )
    ''', mydb)
    

def initSummaryStatisticTables():
    # define empty columns to store summary statistics
    summary_statistics1_radiology = pd.DataFrame(data={'DIAGNOSIS_CODE':[], 
                       'PHENOTYPE':[], 
                       'DIAGNOSIS_VALUE':[], 
                       'PHENOTYPE_VALUE':[], 
                       'N':[]},
                columns = ['DIAGNOSIS_CODE', 'PHENOTYPE', 'DIAGNOSIS_VALUE', 'PHENOTYPE_VALUE', 'N'])
    
    summary_statistics1_lab = pd.DataFrame(data={'DIAGNOSIS_CODE':[], 
                       'PHENOTYPE':[], 
                       'DIAGNOSIS_VALUE':[], 
                       'PHENOTYPE_VALUE':[], 
                       'N':[]},
                columns = ['DIAGNOSIS_CODE', 'PHENOTYPE', 'DIAGNOSIS_VALUE', 'PHENOTYPE_VALUE', 'N'])

    summary_statistics2 = pd.DataFrame(data={'DIAGNOSIS_CODE':[], 
                       'PHEN_TXT':[], 
                       'PHEN_LAB':[], 
                       'DIAGNOSIS_VALUE':[], 
                       'PHEN_TXT_VALUE':[], 
                       'PHEN_LAB_VALUE':[], 
                       'N':[]},
                columns = ['DIAGNOSIS_CODE', 'PHEN_TXT', 'PHEN_LAB', 'DIAGNOSIS_VALUE', 'PHEN_TXT_VALUE', 'PHEN_LAB_VALUE', 'N']) 

    return summary_statistics1_radiology, summary_statistics1_lab, summary_statistics2

def initTables(debug=False):
    """
    This combines LabHpo and Inferred_LabHpo, and combines TextHpo and Inferred_TextHpo. 
    Only need to run once. For efficiency consideration, the tables can also be created as perminent. 
    It is time-consuming, so call it with caution. 
    """
    #init textHpoProfile and index it
    #I create perminant tables to save time; other users should enable them
    #textHpoProfile(include_inferred=True, threshold=1)
    #indexTextHpoProfile()
    #init labHpoProfile and index it
    #labHpoProfile(threshold=1, include_inferred=True, force_update=True)
    #indexLabHpoProfile()
    
    #define encounters to analyze
    encounterOfInterest(debug)
    #init diagnosisProfile
    diagnosisProfile()
    

def iterate(diagnosis_threshold_min, textHpo_threshold_min, labHpo_threshold_min, logger): 
    logger.info('starting iterating...................................')
    N = pd.read_sql_query("SELECT count(*) FROM JAX_encounterOfInterest", mydb)
    # init empty tables to hold summary statistics
    summary_statistics1_radiology, summary_statistics1_lab, summary_statistics2 = initSummaryStatisticTables()
    
    # define a set of diseases that we want to analyze
    rankICD()
    
    diseaseOfInterest = pd.read_sql_query("SELECT * FROM JAX_diagFrequencyRank WHERE N > {}".format(diagnosis_threshold_min), mydb).ICD9_CODE.values
    diseaseOfInterest = ['428']
    # define encounters to analyze
    logger.info('diseases of interest established: {}'.format(len(diseaseOfInterest)))
    for diagnosis in diseaseOfInterest:
        logger.info("start analyzing disease {}".format(diagnosis))
        
        # assign each encounter whether a diagnosis code is observed
        # create a table j1 (joint 1)
        createDiagnosisTable(diagnosis)
        # for every diagnosis, find phenotypes of interest to look at from radiology reports
        # for every diagnosis, find phenotypes of interest to look at from laboratory tests
        rankHpoFromText(diagnosis)
        rankHpoFromLab(diagnosis)
        
        textHpoOfInterest = pd.read_sql_query("SELECT * FROM JAX_textHpoFrequencyRank WHERE N > {}".format(textHpo_threshold_min), mydb).MAP_TO.values
        labHpoOfInterest = pd.read_sql_query("SELECT * FROM JAX_labHpoFrequencyRank WHERE N > {}".format(labHpo_threshold_min), mydb).MAP_TO.values
        logger.info("TextHpo of interest established, size: {}".format(len(textHpoOfInterest)))
        logger.info("LabHpo of interest established, size: {}".format(len(labHpoOfInterest)))
        for textHpo in textHpoOfInterest:
            logger.info("iteration: TextHpo--{}".format(textHpo))
            # assign each encounter whether a phenotype is observed from radiology reports
            diagnosisTextHpo(textHpo)            
            result1_text = pd.read_sql_query('''
                SELECT 
                    '{}' AS DIAGNOSIS_CODE, '{}' AS PHENOTYPE, DIAGNOSIS AS DIAGNOSIS_VALUE, PHEN_TXT_VALUE AS PHENOTYPE_VALUE, COUNT(*) AS N 
                FROM JAX_mf_diag_textHpo 
                GROUP BY 
                    DIAGNOSIS, PHEN_TXT_VALUE
            '''.format(diagnosis, textHpo), mydb)
            summary_statistics1_radiology = summary_statistics1_radiology.append(result1_text)
            # summary statistics for p1
            # calculate I(p1;D)
            for labHpo in labHpoOfInterest:
                logger.info(".........LabHpo--{}".format(labHpo))
                diagnosisLabHpo(labHpo)
                result1_lab = pd.read_sql_query('''
                    SELECT 
                        '{}' AS DIAGNOSIS_CODE, '{}' AS PHENOTYPE, DIAGNOSIS AS DIAGNOSIS_VALUE, PHEN_LAB_VALUE AS PHENOTYPE_VALUE, COUNT(*) AS N 
                    FROM 
                        JAX_mf_diag_labHpo 
                    GROUP BY DIAGNOSIS, PHEN_LAB_VALUE
                '''.format(diagnosis, labHpo), mydb)
                summary_statistics1_lab = summary_statistics1_lab.append(result1_lab)
            
                # assign each encounter whether a phenotype is observed from lab tests
                diagnosisTextLab(labHpo)
                result2 = pd.read_sql_query('''
                    SELECT 
                        '{}' AS DIAGNOSIS_CODE, 
                        '{}' AS PHEN_TXT, 
                        '{}' AS PHEN_LAB,  
                        DIAGNOSIS AS DIAGNOSIS_VALUE, 
                        PHEN_TXT_VALUE, 
                        PHEN_LAB_VALUE, 
                        COUNT(*) AS N
                    FROM JAX_mf_diag_txtHpo_labHpo 
                    GROUP BY DIAGNOSIS, PHEN_TXT_VALUE, PHEN_LAB_VALUE
                '''.format(diagnosis, textHpo, labHpo), mydb)
                summary_statistics2 = summary_statistics2.append(result2)
    logger.info('end iterating.....................................')            
    return N, summary_statistics1_radiology, summary_statistics1_lab, summary_statistics2              

In [156]:
# how to run this

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

# 1. build the temp tables for Lab converted HPO, Text convert HPO
# Read the comments within the method!
initTables(debug=True)
# 2. iterate the database t (for debug, use parameter values: 0, 10, 15, for production, use parameter values: 0, 10000, 10000
N, summary_statistics1_radiology, summary_statistics1_lab, summary_statistics2 = iterate(diagnosis_threshold_min=0, textHpo_threshold_min=10, labHpo_threshold_min=15, logger=logger)
#N, summary_statistics1_radiology, summary_statistics1_lab, summary_statistics2 = iterate(diagnosis_threshold_min=0, textHpo_threshold_min=1000, labHpo_threshold_min=1000, logger=logger)

2019-09-19 16:17:50,748 - 2590 - root - INFO - starting iterating...................................
2019-09-19 16:17:50,762 - 2590 - root - INFO - diseases of interest established: 1
2019-09-19 16:17:50,763 - 2590 - root - INFO - start analyzing disease 428
2019-09-19 16:17:50,776 - 2590 - root - INFO - TextHpo of interest established, size: 5
2019-09-19 16:17:50,777 - 2590 - root - INFO - LabHpo of interest established, size: 7
2019-09-19 16:17:50,778 - 2590 - root - INFO - iteration: TextHpo--HP:0000118
2019-09-19 16:17:50,785 - 2590 - root - INFO - .........LabHpo--HP:0000118
2019-09-19 16:17:50,796 - 2590 - root - INFO - .........LabHpo--HP:0000001
2019-09-19 16:17:50,808 - 2590 - root - INFO - .........LabHpo--HP:0031850
2019-09-19 16:17:50,820 - 2590 - root - INFO - .........LabHpo--HP:0001877
2019-09-19 16:17:50,833 - 2590 - root - INFO - .........LabHpo--HP:0001871
2019-09-19 16:17:50,846 - 2590 - root - INFO - .........LabHpo--HP:0020061
2019-09-19 16:17:50,858 - 2590 - root 

In [144]:
N

,count(*)
0,58976


In [147]:
summary_statistics1_radiology
#summary_statistics1_radiology.groupby('PHENOTYPE').agg({'N': sum})

,DIAGNOSIS_CODE,PHENOTYPE,DIAGNOSIS_VALUE,PHENOTYPE_VALUE,N
0,428,HP:0000001,0,0,7288.0
1,428,HP:0000001,1,1,13467.0
2,428,HP:0000001,0,1,38080.0
3,428,HP:0000001,1,0,141.0
0,428,HP:0000118,0,0,7288.0
1,428,HP:0000118,1,1,13467.0
2,428,HP:0000118,0,1,38080.0
3,428,HP:0000118,1,0,141.0
0,428,HP:0002086,0,0,17412.0
1,428,HP:0002086,1,1,11089.0


In [146]:
summary_statistics2.groupby(['PHEN_TXT', 'PHEN_LAB']).agg({'N': sum})

N
PHEN_TXT   PHEN_LAB           
HP:0000001 HP:0000001  58976.0
           HP:0000118  58976.0
           HP:0001871  58976.0
           HP:0001877  58976.0
           HP:0001939  58976.0
...                        ...
HP:0012252 HP:0012337  58976.0
           HP:0020058  58976.0
           HP:0020061  58976.0
           HP:0031850  58976.0
           HP:0032180  58976.0

[65 rows x 1 columns]

In [204]:
encounterOfInterest(debug=True)
diagnosisProfile()
rankICD()
diagnosis = '401'
rankHpoFromText(diagnosis)
rankHpoFromLab(diagnosis)

In [205]:
pd.read_sql_query("SELECT * FROM JAX_labHpoFrequencyRank LIMIT 4", mydb)

,MAP_TO,N,PHENOTYPE
0,HP:0001939,30,1
1,HP:0000118,30,1
2,HP:0000001,30,1
3,HP:0001871,30,1


In [159]:
pd.read_sql_query("SELECT * FROM JAX_textHpoFrequencyRank LIMIT 4", mydb)

,MAP_TO,N,PHENOTYPE
0,HP:0000118,31,1
1,HP:0000001,31,1
2,HP:0002086,23,1
3,HP:0002088,20,1


In [111]:
createDiagnosisTable(diagnosis)
pd.read_sql_query("SELECT * FROM JAX_mf_diag", mydb)

,SUBJECT_ID,HADM_ID,DIAGNOSIS
0,2,163353,0
1,3,145834,0
2,4,185777,0
3,5,178980,0
4,6,107064,0
...,...,...,...
95,94,183686,1
96,94,140037,1
97,95,160891,0
98,96,170324,0


In [193]:
diagnosisTextHpo(phenotype='HP:0002086')
result = pd.read_sql_query("SELECT '{}' AS DIAGNOSIS_CODE, '{}' AS PHENOTYPE, DIAGNOSIS AS DIAGNOSIS_VALUE, PHEN_TXT_VALUE AS PHENOTYPE_VALUE, COUNT(*) AS N FROM JAX_mf_diag_textHpo GROUP BY DIAGNOSIS, PHEN_TXT_VALUE", mydb)
result.head()
#result.groupby(['DIAGNOSIS', 'PHEN_TXT_VALUE']).

,DIAGNOSIS_CODE,PHENOTYPE,DIAGNOSIS_VALUE,PHENOTYPE_VALUE,N
0,{},{},0,0,35
1,{},{},1,1,16
2,{},{},0,1,48
3,{},{},1,0,1


In [206]:
diagnosisAllTextHpo(0, 100)
pd.read_sql_query("SELECT * FROM JAX_mf_diag_allTextHpo WHERE PHEN_TXT_VALUE = 1", mydb)
result = pd.read_sql_query("SELECT '{}' AS DIAGNOSIS_CODE, PHEN_TXT AS PHENOTYPE, DIAGNOSIS AS DIAGNOSIS_VALUE, PHEN_TXT_VALUE AS PHENOTYPE_VALUE, COUNT(*) AS N FROM JAX_mf_diag_allTextHpo GROUP BY DIAGNOSIS, PHEN_TXT, PHEN_TXT_VALUE", mydb)
result.groupby(['PHENOTYPE']).agg({'N':sum})
#result

,N
PHENOTYPE,
HP:0000001,100
HP:0000077,100
HP:0000079,100
HP:0000083,100
HP:0000118,100
...,...
HP:0100749,100
HP:0100750,100
HP:0100763,100


In [126]:
summary_statistics1_radiology

,DIAGNOSIS_CODE,PHENOTYPE,DIAGNOSIS_VALUE,PHENOTYPE_VALUE,N


In [134]:
diagnosisLabHpo(phenotype='HP:0002086')

#"""
pd.read_sql_query('''
                    SELECT 
                        '{}' AS DIAGNOSIS_CODE, '{}' AS PHENOTYPE, DIAGNOSIS AS DIAGNOSIS_VALUE, PHEN_LAB_VALUE AS PHENOTYPE_VALUE, COUNT(*) AS N 
                    FROM 
                        JAX_mf_diag_labHpo 
                    GROUP BY DIAGNOSIS, PHEN_LAB_VALUE
                '''.format('401', 'HP:0002086'), mydb)
#"""
#pd.read_sql_query("SELECT * FROM JAX_mf_diag_labHpo", mydb)

,DIAGNOSIS_CODE,PHENOTYPE,DIAGNOSIS_VALUE,PHENOTYPE_VALUE,N
0,401,HP:0002086,0,0,44
1,401,HP:0002086,0,1,25
2,401,HP:0002086,1,1,19
3,401,HP:0002086,1,0,12


In [207]:
diagnosisAllLabHpo(0, 100)
pd.read_sql_query("SELECT * FROM JAX_mf_diag_allLabHpo", mydb)
result = pd.read_sql_query("SELECT '{}' AS DIAGNOSIS_CODE, PHEN_LAB AS PHENOTYPE, DIAGNOSIS AS DIAGNOSIS_VALUE, PHEN_LAB_VALUE AS PHENOTYPE_VALUE, COUNT(*) AS N FROM JAX_mf_diag_allLabHpo GROUP BY DIAGNOSIS, PHEN_LAB, PHEN_LAB_VALUE", mydb)
result.groupby(['PHENOTYPE']).agg({'N':sum})
#result

,N
PHENOTYPE,
HP:0000001,100
HP:0000077,100
HP:0000079,100
HP:0000118,100
HP:0000119,100
...,...
HP:0410174,100
HP:0500015,100
HP:0500020,100


In [137]:
diagnosisTextLab(phenotype='HP:0002086')
#pd.read_sql_query("SELECT * FROM JAX_mf_diag_txtHpo_labHpo", mydb)
pd.read_sql_query('''
                    SELECT 
                        '{}' AS DIAGNOSIS_CODE, 
                        '{}' AS PHEN_TXT, 
                        '{}' AS PHEN_LAB,  
                        DIAGNOSIS AS DIAGNOSIS_VALUE, 
                        PHEN_TXT_VALUE, 
                        PHEN_LAB_VALUE, 
                        COUNT(*) AS N
                    FROM JAX_mf_diag_txtHpo_labHpo 
                    GROUP BY DIAGNOSIS, PHEN_TXT_VALUE, PHEN_LAB_VALUE
                '''.format('401', 'HP:001', 'HP:002'), mydb)

,DIAGNOSIS_CODE,PHEN_TXT,PHEN_LAB,DIAGNOSIS_VALUE,PHEN_TXT_VALUE,PHEN_LAB_VALUE,N
0,401,HP:001,HP:002,0,0,0,26
1,401,HP:001,HP:002,0,1,0,18
2,401,HP:001,HP:002,0,1,1,23
3,401,HP:001,HP:002,1,1,1,15
4,401,HP:001,HP:002,1,0,1,4
5,401,HP:001,HP:002,0,0,1,2
6,401,HP:001,HP:002,1,1,0,8
7,401,HP:001,HP:002,1,0,0,4


In [210]:
diagnosisAllTextAllLab(0, 100)
pd.read_sql_query("SELECT * FROM JAX_mf_diag_allTxtHpo_allLabHpo", mydb)
result = pd.read_sql_query("SELECT '{}' AS DIAGNOSIS_CODE, PHEN_TXT, PHEN_LAB, DIAGNOSIS AS DIAGNOSIS_VALUE,PHEN_TXT_VALUE, PHEN_LAB_VALUE, COUNT(*) AS N FROM JAX_mf_diag_allTxtHpo_allLabHpo GROUP BY DIAGNOSIS, PHEN_LAB, PHEN_LAB_VALUE, PHEN_TXT, PHEN_TXT_VALUE", mydb)
result.groupby(['PHEN_TXT', 'PHEN_LAB']).agg({'N':sum})
#result

DatabaseError: 1137 (HY000): Can't reopen table: 'JAX_mf_diag_allTxtHpo_allLabHpo'

In [124]:
summary_statistics1_radiology, summary_statistics1_lab, summary_statistics2=initSummaryStatisticTables()

In [135]:
summary_statistics2

,DIAGNOSIS_CODE,PHEN_TXT,PHEN_LAB,DIAGNOSIS_VALUE,PHEN_TXT_VALUE,PHEN_LAB_VALUE,N


In [149]:
cursor.close()
mydb.close()